# Determining the target audience for the restaurant

In this section, we will try to determine target ethnic group and and according to that ethnic gorup which cusine we will have in our restaurant. 

Installing required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import time

ModuleNotFoundError: No module named 'folium'

Intalling requirments and Population data from Wikipedia

In [2]:
!wget 'https://en.wikipedia.org/wiki/Demographics_of_Toronto' 'Toronto_population.json' 
print('Data downloaded!')
!pip install folium
!pip install lxml
!pip install html5lib
!pip install bs4

--2021-01-28 07:08:31--  https://en.wikipedia.org/wiki/Demographics_of_Toronto
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Demographics_of_Toronto’

Demographics_of_Tor     [ <=>                ] 332.09K  --.-KB/s    in 0.03s   

2021-01-28 07:08:32 (10.5 MB/s) - ‘Demographics_of_Toronto’ saved [340065]

--2021-01-28 07:08:32--  http://toronto_population.json/
Resolving toronto_population.json (toronto_population.json)... failed: Name or service not known.
wget: unable to resolve host address ‘toronto_population.json’
FINISHED --2021-01-28 07:08:32--
Total wall clock time: 0.4s
Downloaded: 1 files, 332K in 0.03s (10.5 MB/s)
Data downloaded!
     |████████████████████████████████| 94 kB 6.5 MB/s  eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=12

Scraping population data from Wikipedia 

In [96]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
 
url_cntr = 'https://en.wikipedia.org/wiki/Demographics_of_Toronto'
t = requests.get(url_cntr)
html_content = t.content
html_soup = BeautifulSoup(html_content, 'html.parser')
sov_tables = html_soup.find('table', class_="wikitable sortable")
df = pd.read_html(str(sov_tables))
df = df[0]
df

Visible minority and Aboriginal population[30][31][32]  \
                                         Population group   
0                                    European (White)       
1                              Visible minority group       
2                              Visible minority group       
3                              Visible minority group       
4                              Visible minority group       
5                              Visible minority group       
6                              Visible minority group       
7                              Visible minority group       
8                              Visible minority group       
9                              Visible minority group       
10                             Visible minority group       
11                             Visible minority group       
12                             Visible minority group       
13                  Total visible minority population       
14                                   Aboriginal group       
15                                   Aboriginal group       
16                                   Aboriginal group       
17                                   Aboriginal group       
18                                   Aboriginal group       
19                        Total Aboriginal population       
20                                   Total population       

                                                         \
                   Population group.1 Population (2016)   
0                    European (White)           1282750   
1                         South Asian            338965   
2                             Chinese            299460   
3                               Black            239850   
4                            Filipino            152715   
5                      Latin American             77160   
6                                Arab             36030   
7                     Southeast Asian             41645   
8                          West Asian             60325   
9                              Korean             41640   
10                           Japanese             13410   
11           Visible minority, n.i.e.             36975   
12        Multiple visible minorities             47675   
13  Total visible minority population           1385850   
14                      First Nations             14380   
15                              Métis              7270   
16                              Inuit               275   
17                 Aboriginal, n.i.e.               645   
18     Multiple Aboriginal identities               500   
19        Total Aboriginal population             23065   
20                   Total population           2691665   

                                                   \
   % of total population (2016) Population (2011)   
0                           NaN           1292365   
1                           NaN            317100   
2                           NaN            278390   
3                           NaN            218160   
4                           NaN            132445   
5                           NaN             71205   
6                           NaN             28920   
7                           NaN             46825   
8                           NaN             50235   
9                           NaN             37225   
10                          NaN             12315   
11                          NaN             33670   
12                          NaN             37920   
13                          NaN           1264395   
14                          NaN             12990   
15                          NaN              4875   
16                          NaN               305   
17                          NaN               920   
18                          NaN               180   
19                          NaN             19265   
20                         100%           2576025   

                     

Cleaning columns

In [112]:
df1 = df["Visible minority and Aboriginal population[30][31][32]"]
df2 = df1.drop(['Population group','% of total population (2016)','% of total population (2011)','Population (2006)','% of total population (2006)'], axis=1)
df2

,Population group.1,Population (2016),Population (2011)
0,European (White),1282750,1292365
1,South Asian,338965,317100
2,Chinese,299460,278390
3,Black,239850,218160
4,Filipino,152715,132445
5,Latin American,77160,71205
6,Arab,36030,28920
7,Southeast Asian,41645,46825
8,West Asian,60325,50235
9,Korean,41640,37225


Removing Total rows and Multiple ethnic groups which is not clearly point our a specific ethnic group (except Total population as I am going to use it in order to calculate population percentage of the entire population)   in the table

In [122]:
df2 = df2[df2['Population group.1'] != 'Total Aboriginal population']
df2 = df2[df2['Population group.1'] != 'Total visible minority population']
df2 = df2[df2['Population group.1'] != 'Multiple Aboriginal identities']
df2 = df2[df2['Population group.1'] != 'Multiple visible minorities']
df2 = df2[df2['Population group.1'] != 'Visible minority, n.i.e.']
df2 = df2[df2['Population group.1'] != 'Aboriginal, n.i.e.']
df2

,Population group.1,Population (2016),Population (2011)
0,European (White),1282750,1292365
1,South Asian,338965,317100
2,Chinese,299460,278390
3,Black,239850,218160
4,Filipino,152715,132445
5,Latin American,77160,71205
6,Arab,36030,28920
7,Southeast Asian,41645,46825
8,West Asian,60325,50235
9,Korean,41640,37225


Renaming Population Group Column

In [132]:
df3=df2.rename(columns={'Population group.1': 'Ethnic Group'})

Adding percentage increase in population in order to determine target ethnic group (which increases most)

In [133]:
df3['Increase_In_Percentage'] = (((df3['Population (2016)'] - df3['Population (2011)']) / df3['Population (2011)']) * 100).round(2)
df3['Percentage_In_Total_Population2016'] = ((df3['Population (2016)'] / 2691665) * 100).round(2)
df3

,Ethnic Group,Population (2016),Population (2011),Increase_In_Percentage,Percentage_In_Total_Population2016
0,European (White),1282750,1292365,-0.74,47.66
1,South Asian,338965,317100,6.90,12.59
2,Chinese,299460,278390,7.57,11.13
3,Black,239850,218160,9.94,8.91
4,Filipino,152715,132445,15.30,5.67
5,Latin American,77160,71205,8.36,2.87
6,Arab,36030,28920,24.59,1.34
7,Southeast Asian,41645,46825,-11.06,1.55
8,West Asian,60325,50235,20.09,2.24
9,Korean,41640,37225,11.86,1.55


We can now remove total population form our list

In [134]:
df3 = df3[df3['Ethnic Group'] != 'Total population']

Let`s sort the list by increase in population percentage and list top 5

In [138]:
sorted_df = df3.sort_values(by='Increase_In_Percentage', ascending=False)
sorted_df.head(5)

,Ethnic Group,Population (2016),Population (2011),Increase_In_Percentage,Percentage_In_Total_Population2016
15,Métis,7270,4875,49.13,0.27
6,Arab,36030,28920,24.59,1.34
8,West Asian,60325,50235,20.09,2.24
4,Filipino,152715,132445,15.30,5.67
9,Korean,41640,37225,11.86,1.55


# Conclusion of determination of type of restaurant cousine

In order to determine target ethnic group we need to evaluate 2 data (Increase in percentage and Percenge in total population) at the same time.
As it can be observed from the list, although the increase in poupaltion percentage is highest in Metis however percentage in total population is too low. 

On the other side, Arab's and West Asians cant be selected as they are not specific ethnic groups and it is not easy to determine the kitchen. 
Conclusion, Filipino is the highest percentage in total population in our top 5. Lets decide where to locate the restaurant. 
We will use our previous Toronto exercise to determine where to locate our restaurant.   

# Data analysis for Toronto Venues using Foursquare

we will follow the same step which we did throughout the Toronto exercise and we will try to determine which location will be most suitable for Filipino restaurant

In [139]:
!wget 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 'Toronto_data.json' 
print('Data downloaded!')
!pip install lxml
!pip install html5lib
!pip install bs4

--2021-01-28 10:18:31--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54650 (53K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.37K  --.-KB/s    in 0.01s   

2021-01-28 10:18:31 (5.05 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54650/54650]

--2021-01-28 10:18:31--  http://toronto_data.json/
Resolving toronto_data.json (toronto_data.json)... failed: Name or service not known.
wget: unable to resolve host address ‘toronto_data.json’
FINISHED --2021-01-28 10:18:31--
Total wall clock time: 0.3s
Downloaded: 1 files, 53K in 0.01s (5.05 MB/s)
Data downloaded!


Scraping data

In [140]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)[0]
df = df[df.Borough != 'Not assigned']

df_postcode = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_postcode

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [141]:
!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df_postcode['Postal Code'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

Postal Code M1B has been retrieved. 102 codes left
Postal Code M1C has been retrieved. 101 codes left
Postal Code M1E has been retrieved. 100 codes left
Postal Code M1G has been retrieved. 99 codes left
Postal Code M1H has been retrieved. 98 codes left
Postal Code M1J has been retrieved. 97 codes left
Postal Code M1K has been retrieved. 96 codes left
Postal Code M1L has been retrieved. 95 codes left
Postal Code M1M has been retrieved. 94 codes left
Postal Code M1N has been retrieved. 93 codes left
Postal Code M1P has been retrieved. 92 codes left
Postal Code M1R has been retrieved. 91 codes left
Postal Code M1S has been retrieved. 90 codes left
Postal Code M1T has been retrieved. 89 codes left
Postal Code M1V has been retrieved. 88 codes left
Postal Code M1W has been retrieved. 87 codes left
Postal Code M1X has been retrieved. 86 codes left
Postal Code M2H has been retrieved. 85 codes left
Postal Code M2J has been retrieved. 84 codes left
Postal Code M2K has been retrieved. 83 codes le

In [142]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
df_wcoordinates = pd.merge(df_postcode, coordinates, 'left', 'Postal Code')

Getting Toronto coordinates

In [143]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.6708625, -79.37279241253721.


Mapping the neighbourhoods

In [145]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_wcoordinates['Latitude'], df_wcoordinates['Longitude'], df_wcoordinates['Borough'], df_wcoordinates['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get the neighborhood's latitude and longitude values

In [147]:
CLIENT_ID = 'ZOBU3LMQ210JGDHJSJFMA0CC0DIIOXJAHVZQA0C4OTYMOXTH' # your Foursquare ID
CLIENT_SECRET = 'IF2BSW343V1KVB2LOCDDWE1AB0X44LJYJYYFAVBDPR204OQC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

df_wcoordinates.loc[0, 'Neighbourhood']

neighborhood_latitude = df_wcoordinates.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_wcoordinates.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_wcoordinates.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [148]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60129cb4b1415a5dbf8fa2b3'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80228301948931,
          'lng': -79.19856472801668}],
       

Lets get the categories

In [149]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [150]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,RBC Royal Bank,Bank,43.798782,-79.197090
4,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [151]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Code to run the above function on each neighborhood and create a new dataframe called toronto_venues

In [152]:
toronto_venues = getNearbyVenues(names=df_wcoordinates['Neighbourhood'],
                                   latitudes=df_wcoordinates['Latitude'],
                                   longitudes=df_wcoordinates['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

how many venues were returned for each neighborhood

In [169]:
a = toronto_venues.groupby('Neighborhood').count()
a

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


Lets analyze each neighbourhood

In [154]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00

lets analyze whether we have any Filipino restaurant 

In [157]:
toronto_grouped2= toronto_grouped[toronto_grouped['Filipino Restaurant'] != 0]
toronto_grouped2

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
43,"Kensington Market, Chinatown, Grange Park",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.0,0.0,0.0,0.0,0.0158

As per above check, we have only 1 neighborhood with Filipino restaurant. This is quiet interesting that there is only 1 neighborhood with Filipino restaurant where tehre are a lot of increase in Filipino population in the city 

Now lets check how many venues in that location in order to understand the intensity of venues in that neighborhood

# Conclusion of determined location and  suitablility check 

In [173]:
toronto_venues2 = toronto_venues[toronto_venues['Neighborhood'] == 'Kensington Market, Chinatown, Grange Park']
average_venue= a['Venue'].mean()
average_venue
print('Total venue number is', toronto_venues2['Venue'].count())
print('Average venue for neighborhood  is', average_venue)


Total venue number is 63
Average venue for neighborhood  is 22.020833333333332


Kensington Market, Chinatown, Grange Park is a quiet intense neighborhood with 63 venues while compared to average venue per neighborhood is 22. 